# Model Input Data Preparation

In this notebook, we will prepare five essential files to construct a foundational class called `DroughtDataset`, based on the `TabularDataset` class from the [torch-spatiotemporal library](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/datasets_prototypes.html#tsl.datasets.prototypes.TabularDataset). 

In the next notebook (`6.training.ipynb`), we will utilize this class to create a proper structure for feeding data into deep learning models.

The files we will prepare include:

- **target**: Stores our target variable data.
- **mask**: Specifies areas or time periods with missing or irrelevant data.
- **covariates**: Contains feature data that will serve as model inputs.
- **metadata**: Provides additional information about each data point, such as location and time-specific attributes.
- **distance matrix**: Contains distances between spatial points, which supports spatial relationship modeling.


## Load data

In [1]:
import xarray as xr

# Load the dataset
ds = xr.open_dataset(
    'ml-drought-forecasting/soil-water-forecasting/data/04_feature/features.nc'
)

In [2]:
ds

<xarray.Dataset> Size: 1GB
Dimensions:    (date: 96, latitude: 181, longitude: 360)
Coordinates:
    number     int64 8B ...
  * date       (date) datetime64[ns] 768B 2016-01-01 2016-02-01 ... 2023-12-01
    expver     (date) <U4 2kB ...
  * latitude   (latitude) float64 1kB -90.0 -89.0 -88.0 -87.0 ... 88.0 89.0 90.0
  * longitude  (longitude) int64 3kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
Data variables: (12/24)
    u10        (date, latitude, longitude) float64 50MB ...
    v10        (date, latitude, longitude) float64 50MB ...
    t2m        (date, latitude, longitude) float64 50MB ...
    sst        (date, latitude, longitude) float64 50MB ...
    sp         (date, latitude, longitude) float64 50MB ...
    tp         (date, latitude, longitude) float64 50MB ...
    ...         ...
    cvh        (date, latitude, longitude) float64 50MB ...
    lai_hv     (date, latitude, longitude) float64 50MB ...
    tvh        (date, latitude, longitude) float64 50MB ...
    z          (date, latitude, longitude) float64 50MB ...
    lsm        (date, latitude, longitude) float64 50MB ...
    season     (date, latitude, longitude) int32 25MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-16T11:34 GRIB to CDM+CF via cfgrib-0.9.1...

## Target

The `target` stores values for the target variable, `swvl1`, over time and location.



In [3]:
import numpy as np

# Define the target variables to extract from the dataset
targets = ['swvl1']
data_arrays = [ds[var] for var in targets]

# Flatten latitude and longitude into a single "node" dimension
# This combines the latitude and longitude coordinates as one "node" dimension
data_arrays_flattened = [da.stack(node=('latitude', 'longitude')) for da in data_arrays]

# Convert each DataArray to a numpy array and add a new channel dimension
# This results in an array with shape (time, nodes, channels) for the target data
target = np.stack([da.to_numpy() for da in data_arrays_flattened], axis=-1)


In [4]:
target

array([[[ 1.64963067e-01],
        [ 1.64963067e-01],
        [ 1.64963067e-01],
        ...,
        [ 2.92435288e-07],
        [ 2.92435288e-07],
        [ 2.92435288e-07]],

       [[ 1.66875362e-01],
        [ 1.66875362e-01],
        [ 1.66875362e-01],
        ...,
        [ 5.25265932e-06],
        [ 5.25265932e-06],
        [ 5.25265932e-06]],

       [[ 1.67241454e-01],
        [ 1.67241454e-01],
        [ 1.67241454e-01],
        ...,
        [-1.01362821e-05],
        [-1.01362821e-05],
        [-1.01362821e-05]],

       ...,

       [[ 3.21812391e-01],
        [ 3.21812391e-01],
        [ 3.21812391e-01],
        ...,
        [ 4.52739187e-06],
        [ 4.52739187e-06],
        [ 4.52739187e-06]],

       [[ 3.26855004e-01],
        [ 3.26855004e-01],
        [ 3.26855004e-01],
        ...,
        [-3.52994539e-06],
        [-3.52994539e-06],
        [-3.52994539e-06]],

       [[ 3.28729928e-01],
        [ 3.28729928e-01],
        [ 3.28729928e-01],
        ...,
        

In [5]:
# Save as .npy file
np.save('ml-drought-forecasting/soil-water-forecasting/data/05_model_input/target.npy', target)

## Mask 

Boolean mask denoting if values in target are valid (True) or not (False).

In [6]:
mask = np.where(~np.isnan(target), 1, 0)

In [7]:
mask

array([[[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       ...,

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]]])

In [8]:
# Save as .npy file
np.save('ml-drought-forecasting/soil-water-forecasting/data/05_model_input/mask.npy', mask)

## Covariates 

The `covariates` stores values for the features variables over time and location.

In [9]:
import numpy as np

# Define covariates: Relevant variables from the dataset to be used as features in the model
variables = [
    'u10',             # 10-meter U wind component
    'v10',             # 10-meter V wind component
    't2m',             # 2-meter temperature
    'sst',             # Sea surface temperature
    'sp',              # Surface pressure
    'tp',              # Total precipitation
    'ssr',             # Surface solar radiation
    'ssrd',            # Surface solar radiation downward
    'tcc',             # Total cloud cover
    'cl',              # Cloud liquid water content
    'e',               # Evaporation
    'pev',             # Potential evaporation
    'ro',              # Runoff
    'asn',             # Snowfall amount
    'rsn',             # Rainfall amount
    'sd',              # Snow depth
    'slt',             # Soil type
    # 'swvl1',           # Soil water in layer 1 - target 
    'cvh',             # High vegetation cover
    'lai_hv',          # High vegetation leaf area index
    'tvh',             # High vegetation transpiration
    'z',               # Geopotential
    # 'lsm',             # Land-sea mask - dont need as feature 
    'season',          # Season (single temporal feature)
]

# Extract the defined covariate variables from the dataset
data_arrays = [ds[var] for var in variables]

# Flatten latitude and longitude into a single "node" dimension
# This combines the spatial coordinates into one dimension labeled "node"
data_arrays_flattened = [da.stack(node=('latitude', 'longitude')) for da in data_arrays]

# Convert each DataArray to a numpy array and add a new channel dimension
# The resulting array has the shape (time, nodes, channels), ready for model input
covariates = np.stack([da.to_numpy() for da in data_arrays_flattened], axis=-1)

# Replace NaN values with 0 to handle missing data
covariates = np.nan_to_num(covariates, nan=0.0)


In [10]:
covariates

array([[[ 3.05854797e-01, -1.29899025e-01,  2.43355896e+02, ...,
          0.00000000e+00,  2.73551035e+04,  1.00000000e+00],
        [ 3.05854797e-01, -1.29899025e-01,  2.43355896e+02, ...,
          0.00000000e+00,  2.73551035e+04,  1.00000000e+00],
        [ 3.05854797e-01, -1.29899025e-01,  2.43355896e+02, ...,
          0.00000000e+00,  2.73551035e+04,  1.00000000e+00],
        ...,
        [-7.74459839e-02,  3.22103500e-02,  2.50641052e+02, ...,
          0.00000000e+00,  3.22753906e-01,  1.00000000e+00],
        [-7.74459839e-02,  3.22103500e-02,  2.50641052e+02, ...,
          0.00000000e+00,  3.22753906e-01,  1.00000000e+00],
        [-7.74459839e-02,  3.22103500e-02,  2.50641052e+02, ...,
          0.00000000e+00,  3.22753906e-01,  1.00000000e+00]],

       [[ 4.19805527e-01,  1.15586281e-01,  2.33366547e+02, ...,
          0.00000000e+00,  2.73551035e+04,  1.00000000e+00],
        [ 4.19805527e-01,  1.15586281e-01,  2.33366547e+02, ...,
          0.00000000e+00,  2.73551035e

In [11]:
# Save as .npy file
np.save('ml-drought-forecasting/soil-water-forecasting/data/05_model_input/covariates.npy', covariates)

## Metadata

The metadata store values for the latitude and longitude of each specific location (node).







In [12]:
import pandas as pd
import os

def generate_and_save_metadata(df: pd.DataFrame, lat_col: str = 'lat', lon_col: str = 'lon', save_directory: str = None) -> pd.DataFrame:
    """
    Generates metadata from a given DataFrame by identifying unique latitude and longitude
    combinations and assigning a unique node ID to each combination. The metadata is then
    saved to a specified Parquet file.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - lat_col (str): The name of the column containing latitude data.
    - lon_col (str): The name of the column containing longitude data.
    - save_directory (str, optional): The directory where the metadata file will be saved. If None, 
                                      the file will be saved in the current working directory.

    Returns:
    - pd.DataFrame: A DataFrame containing the metadata with latitude, longitude, and 'node_id' as columns.
    """
    # Ensure the DataFrame contains necessary columns
    if lat_col not in df.columns or lon_col not in df.columns:
        raise ValueError(f"DataFrame must contain '{lat_col}' and '{lon_col}' columns.")

    # Create a unique node ID for each unique latitude-longitude combination
    unique_lat_lon = df[[lat_col, lon_col]].drop_duplicates().reset_index(drop=True)
    unique_lat_lon['node_id'] = unique_lat_lon.index

    # Create the metadata DataFrame
    metadata = unique_lat_lon.set_index('node_id')

    # Handle save directory and save metadata to Parquet file
    if save_directory:
        os.makedirs(save_directory, exist_ok=True)  # Create directory if it doesn't exist
        file_path = os.path.join(save_directory, "metadata.parquet")
    else:
        file_path = "metadata.parquet"  # Save in the current working directory

    metadata.to_parquet(file_path)
    print(f"Metadata file saved at: {file_path}")

    return metadata


In [13]:
import pandas as pd
import os

# Assume you've already stacked your data arrays as 'data_arrays_flattened'
# We'll use one of the flattened DataArrays to extract the node information
da_flattened = data_arrays_flattened[0]  # Using the first variable for example

# Get the MultiIndex from the 'node' dimension
node_index = da_flattened.indexes['node']

# Extract latitude and longitude from the MultiIndex
latitudes = node_index.get_level_values('latitude').values
longitudes = node_index.get_level_values('longitude').values

# Create a DataFrame with 'lat' and 'lon' columns
df = pd.DataFrame({
    'lat': latitudes,
    'lon': longitudes
})

# Now use your 'generate_and_save_metadata' function
metadata = generate_and_save_metadata(
    df,
    lat_col='lat',
    lon_col='lon',
    save_directory='ml-drought-forecasting/soil-water-forecasting/data/05_model_input/'
)


Metadata file saved at: ml-drought-forecasting/soil-water-forecasting/data/05_model_input/metadata.parquet


<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.Tensor size changed, may indicate binary incompatibility. Expected 64 from C header, got 80 from PyObject


In [4]:
import pandas as pd
metadata = pd.read_parquet("ml-drought-forecasting/soil-water-forecasting/data/05_model_input/metadata.parquet")

In [5]:
metadata

,lat,lon
node_id,,
0,-90.0,0
1,-90.0,1
2,-90.0,2
3,-90.0,3
4,-90.0,4
...,...,...
65155,90.0,355
65156,90.0,356
65157,90.0,357


In [6]:
# Converting the DataFrame to a numpy ndarray
metadata_array = metadata.to_numpy()

In [8]:
# Save as .npy file
np.save('ml-drought-forecasting/soil-water-forecasting/data/05_model_input/metadata.npy', metadata_array)

## Distance matrix 

The distance_matrix stores values for the geographical distances between locations (nodes).


In [1]:
import pandas as pd
metadata = pd.read_parquet("ml-drought-forecasting/soil-water-forecasting/data/05_model_input/metadata.parquet")

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.Tensor size changed, may indicate binary incompatibility. Expected 64 from C header, got 80 from PyObject


In [2]:
metadata

,lat,lon
node_id,,
0,-90.0,0
1,-90.0,1
2,-90.0,2
3,-90.0,3
4,-90.0,4
...,...,...
65155,90.0,355
65156,90.0,356
65157,90.0,357


In [3]:
from tsl.ops.similarities import geographical_distance
# Calculate geographical distances with coordinates converted to radians.
distance_matrix = geographical_distance(metadata, to_rad=True).values

In [4]:
distance_matrix

array([[0.00000000e+00, 6.80864854e-15, 1.36167786e-14, ...,
        2.00151144e+04, 2.00151144e+04, 2.00151144e+04],
       [6.80864854e-15, 0.00000000e+00, 6.80864854e-15, ...,
        2.00151144e+04, 2.00151144e+04, 2.00151144e+04],
       [1.36167786e-14, 6.80864854e-15, 0.00000000e+00, ...,
        2.00151144e+04, 2.00151144e+04, 2.00151144e+04],
       ...,
       [2.00151144e+04, 2.00151144e+04, 2.00151144e+04, ...,
        0.00000000e+00, 6.80864854e-15, 1.36167786e-14],
       [2.00151144e+04, 2.00151144e+04, 2.00151144e+04, ...,
        6.80864854e-15, 0.00000000e+00, 6.80864854e-15],
       [2.00151144e+04, 2.00151144e+04, 2.00151144e+04, ...,
        1.36167786e-14, 6.80864854e-15, 0.00000000e+00]])

In [5]:
type(distance_matrix)

numpy.ndarray

In [8]:
import numpy as np

# Save as .npy file
np.save('/teamspace/studios/this_studio/ml-drought-forecasting/soil-water-forecasting/data/05_model_input/distance_matrix.npy', distance_matrix)

# Test

Below is a test to verify the functionality of the class.







In [1]:
from typing import Optional, Union, List
import numpy as np
import pandas as pd 

from tsl.datasets.prototypes import TabularDataset
from tsl.ops.similarities import gaussian_kernel

class SoilWaterDataset(TabularDataset):

    similarity_options = {'distance', 'grid'}

    def __init__(self,
                 root: str = None
                 ):

        self.root = root

        # Load data
        target, mask, u, dist, metadata = self.load()

        covariates = {
            'u': (u),
            'metadata' : (metadata),
            'distances': (dist)
        }

        super().__init__(target=target,
                         mask=mask,
                         covariates=covariates,
                         similarity_score='distance',
                         temporal_aggregation='mean',
                         spatial_aggregation='mean',
                         name='SoilWaterDataset')

    def load(self):
        """
        Load data from files.

        Returns:
            tuple: Containing target, mask, covariates, distances, and metadata.
        """
        target_path = f"{self.root}target.npy"
        mask_path = f"{self.root}mask.npy"
        dist_path = f"{self.root}distance_matrix.npy"
        covariates_path = f"{self.root}covariates.npy"
        metadata_path = f"{self.root}metadata.npy"

        target = np.load(target_path)
        mask = np.load(mask_path)
        u = np.load(covariates_path)
        dist = np.load(dist_path)
        metadata = np.load(metadata_path)

        return target, mask, u, dist, metadata


    def compute_similarity(self, method: str, **kwargs):
        """
        Compute similarity matrix based on the specified method.

        Args:
            method (str): The similarity computation method ('distance' or 'correlation').
            **kwargs: Additional keyword arguments for similarity computation.

        Returns:
            numpy.ndarray: Computed similarity matrix.

        Raises:
            ValueError: If an unknown similarity method is provided.
        """
        if method == "distance":
            # Calculate a Gaussian kernel similarity from the distance matrix, using a default or provided 'theta'
            theta = kwargs.get('theta', np.std(self.distances))
            return gaussian_kernel(self.distances, theta=theta)
        elif method == "grid":
            dist = self.distances.copy()
            dist[dist > 16] = np.inf  # keep only grid edges
            theta = kwargs.get('theta', 20)
            return gaussian_kernel(dist, theta=theta)
        else:
            raise ValueError(f"Unknown similarity method: {method}")

In [2]:
dataset = SoilWaterDataset(root='ml-drought-forecasting/soil-water-forecasting/data/05_model_input/')

In [3]:
dataset.distances

array([[0.0000000e+00, 6.8086486e-15, 1.3616779e-14, ..., 2.0015115e+04,
        2.0015115e+04, 2.0015115e+04],
       [6.8086486e-15, 0.0000000e+00, 6.8086486e-15, ..., 2.0015115e+04,
        2.0015115e+04, 2.0015115e+04],
       [1.3616779e-14, 6.8086486e-15, 0.0000000e+00, ..., 2.0015115e+04,
        2.0015115e+04, 2.0015115e+04],
       ...,
       [2.0015115e+04, 2.0015115e+04, 2.0015115e+04, ..., 0.0000000e+00,
        6.8086486e-15, 1.3616779e-14],
       [2.0015115e+04, 2.0015115e+04, 2.0015115e+04, ..., 6.8086486e-15,
        0.0000000e+00, 6.8086486e-15],
       [2.0015115e+04, 2.0015115e+04, 2.0015115e+04, ..., 1.3616779e-14,
        6.8086486e-15, 0.0000000e+00]], dtype=float32)

In [4]:
# Adjust connectivity to reduce the number of edges
connectivity = dataset.get_connectivity(  
    method='distance',
    knn=4,     
    include_self=False,
    force_symmetric=True, 
    layout="edge_index",
)

: 

In [5]:
connectivity

(array([[    0,     0,     0, ..., 65159, 65159, 65159],
        [    1,     2,     3, ..., 65156, 65157, 65158]]),
 array([0.125, 0.125, 0.125, ..., 0.125, 0.125, 0.125], dtype=float32))

In [6]:
edge_index, edge_weight = connectivity

In [7]:
import torch

torch.save({"edge_index": torch.tensor(edge_index),
 "edge_weight": torch.tensor(edge_weight)},
  "ml-drought-forecasting/soil-water-forecasting/data/05_model_input/connectivity.pt")

In [ ]:
import torch

connectivity = torch.load("ml-drought-forecasting/soil-water-forecasting/data/05_model_input/connectivity.pt")
edge_index = connectivity["edge_index"].numpy()
edge_weight = connectivity["edge_weight"].numpy()

In [ ]:
# Pack into the desired tuple format
connectivity = (edge_index, edge_weight)